<a href="https://colab.research.google.com/github/Rathore25/Reddit-Sentiment-Pre-Post-Covid/blob/main/Predict_sentiment_of_subreddits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import json
import codecs

import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import tensorflow as tf
from tensorflow import keras

import time

In [2]:
project_dir = '/content/drive/MyDrive/NEU/6120/Final Project/'
subreddits  = ['boston','losangeles','texas','mentalhealth','depression']
periods     = ["Jan'19","Feb'19","Mar'19","Apr'19","May'19","Jun'19","Jul'19","Aug'19","Sep'19","Oct'19","Nov'19","Dec'19",
               "Jan'20","Feb'20","Mar'20","Apr'20","May'20","Jun'20","Jul'20","Aug'20","Sep'20","Oct'20","Nov'20","Dec'20",
               "Jan'21","Feb'21","Mar'21"]

In [3]:
dfs = {}
for subreddit in subreddits:
  dfs[subreddit] = pd.read_csv('{0}{1}.csv'.format(project_dir,subreddit), encoding = 'utf-8-sig')

In [4]:
#model_train_false_32k
model_path  = project_dir + 'train_false_32k'
model       = keras.models.load_model(model_path)

In [5]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 512)               147354880 
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_7 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 147,502,721
Trainable params: 147,502,721
Non-trainable params: 0
_________________________________________________________________


In [7]:
def getPredictions(df, model):
  print(df.shape)
  batch_size  = 1000
  length      = len(df)
  batches     = length//1000 + 1
  all_texts   = list(df['body'].values)
  predictions = []
  start_time  = time.time()

  for i in range(batches):
    print("Batch:", i)
    start_index = i * batch_size
    end_index   = min(start_index+batch_size,length)
    curr_batch  = all_texts[start_index:end_index]
    curr_pred   = model.predict(curr_batch)
    curr_pred   = np.squeeze(curr_pred)
    predictions.extend(curr_pred)
  
  print("Time taken:", time.time() - start_time)

  return predictions

In [ ]:
for subreddit in subreddits:
  print(subreddit)
  ypred                         = getPredictions(dfs[subreddit], model)
  dfs[subreddit]['prediction']  = ypred
  file_path                     = '{0}{1}_pred.csv'.format(project_dir,subreddit)
  
  with open(file_path, 'w', encoding = 'utf-8-sig') as file:
    file.write(dfs[subreddit].to_csv(index=False,encoding = 'utf-8-sig'))